In [ ]:
!pip install pymystem3

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem
import re
import string

nltk.download('punkt_tab', quiet=True)
nltk.download('averaged_perceptron_tagger_eng', quiet=True)
nltk.download('universal_tagset', quiet=True)

mystem = Mystem()

def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def clean_text(text, language):
    """Удаляет знаки препинания из текста"""
    if language == 'english':
        text = re.sub(r'[^\w\s\']', ' ', text)
        text = re.sub(r'\s+', ' ', text)
    elif language == 'russian':
        ru_punctuation = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~«»—…'
        for char in ru_punctuation:
            text = text.replace(char, ' ')
        text = re.sub(r'\s+', ' ', text)
    return text.strip()

def tokenize_and_tag_english(text):
    tokens = word_tokenize(text)
    tagged_tokens = nltk.pos_tag(tokens, tagset='universal')

    result = []
    for word, tag in tagged_tokens:
        result.append(f"{word.lower()}_{tag}")

    return result

def tokenize_and_tag_russian(text):
    analyzed = mystem.analyze(text)

    result_tokens = []

    for item in analyzed:
        if item.get('text', '').strip() == '' or 'analysis' not in item:
            continue
        token_text = item['text'].lower()

        if 'analysis' in item and item['analysis']:
            first_analysis = item['analysis'][0]

            if 'gr' in first_analysis:
                pos_tag = first_analysis['gr'].split(',')[0].split('=')[0]

                formatted_token = f"{token_text}_{pos_tag}"
                result_tokens.append(formatted_token)
            else:
                result_tokens.append(token_text)
        else:
            result_tokens.append(token_text)

    return result_tokens

def process_text_file(file_path, language):
    print(f"\nОбработка файла: {file_path}")
    print(f"Язык: {language}")

    text = read_file(file_path)
    print(f"Длина исходного текста: {len(text)} символов")
    print(f"Длина исходного текста: {len(text.split())} слов")

    # Очистка текста от знаков препинания
    cleaned_text = clean_text(text, language)
    print(f"Длина текста после очистки: {len(cleaned_text)} символов")
    print(f"Длина текста после очистки: {len(cleaned_text.split())} слов")

    if language == 'english':
        tagged_tokens = tokenize_and_tag_english(cleaned_text)
    elif language == 'russian':
        tagged_tokens = tokenize_and_tag_russian(cleaned_text)
    else:
        print(f"Неизвестный язык: {language}")
        return []

    print(f"Количество размеченных токенов: {len(tagged_tokens)}")

    print("\nПервые 20 размеченных токенов:")
    for i, token in enumerate(tagged_tokens[:20]):
        print(f"{i+1:3}. {token}")

    analyze_pos_distribution(tagged_tokens)

    return tagged_tokens

def analyze_pos_distribution(tagged_tokens):
    pos_counter = {}

    for token in tagged_tokens:
        if '_' in token:
            pos = token.split('_')[-1]
            pos_counter[pos] = pos_counter.get(pos, 0) + 1

    print("\nРаспределение частей речи:")
    total = len(tagged_tokens)
    for pos, count in sorted(pos_counter.items(), key=lambda x: x[1], reverse=True):
        percentage = (count / total) * 100
        print(f"{pos:15}: {count:4} ({percentage:5.1f}%)")

def save_results(tagged_tokens, output_file):
    with open(output_file, 'w', encoding='utf-8') as file:
        for token in tagged_tokens:
            file.write(token + ' ')
    print(f"\nРезультаты сохранены в файл: {output_file}")

if __name__ == "__main__":
    english_tokens = process_text_file('англ.txt', 'english')
    if english_tokens:
        save_results(english_tokens, 'англ_tagged.txt')

    russian_tokens = process_text_file('рус.txt', 'russian')
    if russian_tokens:
        save_results(russian_tokens, 'рус_tagged.txt')

    print("\n" + "="*60)
    print("СРАВНИТЕЛЬНЫЙ АНАЛИЗ")
    print("="*60)

    def count_unique_tokens(tagged_tokens):
        unique = set()
        for token in tagged_tokens:
            word = token.split('_')[0] if '_' in token else token
            unique.add(word)
        return len(unique)

    if english_tokens:
        print(f"\nАнглийский текст:")
        print(f"Всего токенов: {len(english_tokens)}")
        print(f"Уникальных слов: {count_unique_tokens(english_tokens)}")
        print(f"Средняя длина слова: {sum(len(t.split('_')[0]) for t in english_tokens)/len(english_tokens):.1f}")

    if russian_tokens:
        print(f"\nРусский текст:")
        print(f"Всего токенов: {len(russian_tokens)}")
        print(f"Уникальных слов: {count_unique_tokens(russian_tokens)}")
        print(f"Средняя длина слова: {sum(len(t.split('_')[0]) for t in russian_tokens)/len(russian_tokens):.1f}")


Обработка файла: англ.txt
Язык: english
Длина исходного текста: 109935 символов
Длина исходного текста: 15889 слов
Длина текста после очистки: 107748 символов
Длина текста после очистки: 16171 слов
Количество размеченных токенов: 16239

Первые 20 размеченных токенов:
  1. fefu_ADJ
  2. students_NOUN
  3. win_VERB
  4. four_NUM
  5. awards_NOUN
  6. at_ADP
  7. the_DET
  8. acrobatic_NOUN
  9. rock_NOUN
 10. n_CONJ
 11. roll_NOUN
 12. world_NOUN
 13. cup_NOUN
 14. in_ADP
 15. kazakhstan_NOUN
 16. the_DET
 17. team_NOUN
 18. of_ADP
 19. far_NOUN
 20. eastern_NOUN

Распределение частей речи:
NOUN           : 6703 ( 41.3%)
ADP            : 2274 ( 14.0%)
VERB           : 1961 ( 12.1%)
DET            : 1691 ( 10.4%)
ADJ            : 1629 ( 10.0%)
CONJ           :  760 (  4.7%)
ADV            :  340 (  2.1%)
PRT            :  339 (  2.1%)
PRON           :  316 (  1.9%)
NUM            :  213 (  1.3%)
X              :    9 (  0.1%)
.              :    4 (  0.0%)

Результаты сохранены в файл: а